In [ ]:
import os
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 2000)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms
plt.style.use('fivethirtyeight')

In [ ]:
# Import relevant dataframes:

main_table = pd.read_csv(r"C:\Users\Public\_Data\_Data\Latest_Version\Main_Data\Main_Table_10182019.csv")
print(len(main_table))
faculty_table = pd.read_csv(r"C:\Users\Public\_Data\_Data\Latest_Version\Faculty_Data\Faculty_Table_08272019.csv")
print(len(faculty_table))
skill_table = pd.read_csv(r"C:\Users\Public\_Data\_Data\Latest_Version\Skills_Data\Skill_Table_06072019.csv")
print(len(skill_table))

In [ ]:
def title_swap(string):
    dictionary = {'Community and Social Services':'Counseling and Religious Life',
                  'Customer and Client Support':'Online Support and University Information',
                  'Hospitality, Food, and Tourism':'Event Management and Hospitality', 
                  'Planning and Analysis':'Analysis', 
                  'Curriculum and Instructional Designer / Developer':'Curriculum and Instructional Designer', 
                  'Special Education Teacher':'Accessibility and Disability Services', 
                  'Teaching Assistant':'Faculty Support', 
                  'Tutor':'Academic Tutor',
                  'Clerical and Administrative':'Administrative',
                  'na' : 'Uncategorized'}
    if(string in dictionary):
        return dictionary[string]
    else:
        return string
    
def val_year(year1, year2):
    if(year1 >= year2):
        if(year1>year2):
            temp = year1
            year1 = year2
            year2 = temp
        else:
            raise ValueError('Years cannot be the same.')
            
    return (year1, year2)

In [ ]:
mask = skill_table[(skill_table['Skill Name']=='Social Work')|
                   (skill_table['Skill Name']=='Social Services')|
                   (skill_table['Skill Name']=='Policy Analysis')|
                   (skill_table['Skill Name']=='Policy Reserach')|
                   (skill_table['Skill Name']=='Social Studies')|
                   (skill_table['Skill Name']=='Policy Evaluation')|
                   (skill_table['Skill Name']=='Policy Development')][['Job ID']]


#                    &
#                    (skill_table['Skill Name']=='Research')
# Communication Skills
# Teamwork / Collaboration
# Schedulling
# Project Management
# Supervisory Skills
# Budgeting
# Planning
# Grant Writing
# Data Management
# Quantatative Analysis

# to add another filter: | (skill_table['Skill Name']=='')
mask = mask.drop_duplicates('Job ID')
# print(len(mask))

table = main_table.merge(mask, on='Job ID', how='inner')
print('Number of total Jobs:\n\t' + str(len(table)))

table['Career Area'] = table['Career Area'].apply(title_swap)
table['Occupation'] = table['Occupation'].apply(title_swap)

table = table[(table['Minimum EDU Requirements']>=18)]

print('Number of Jobs with MA or Above:\n\t' + str(len(table)))

# print('Number of total Jobs:\n\t', len(table))

mask = faculty_table[(faculty_table['Faculty']==0)&(faculty_table['Post-Doctoral']==0)]['Job ID']
table_non = table.merge(mask, on='Job ID', how='inner')
table_fac = table.merge(mask, on='Job ID', how='outer', indicator=True)
table_fac = table_fac[table_fac['_merge']=='left_only'].drop(columns='_merge')
print('Number of non-faculty Jobs:\n\t' + str(len(table_non)))
print('Number of faculty Jobs:\n\t' + str(len(table_fac)))

# minimum_size = 10
# mask = pd.DataFrame(table['Occupation'].value_counts()).reset_index().rename(columns={'Occupation':'count', 'index':'Occupation'})
# mask = mask[mask['count']>=minimum_size][['Occupation']]
# table = table.merge(mask, on='Occupation', how='inner')
# print('Size of table after removing categories smaller than ' + str(minimum_size) + ' postings:\n\t' +
#       str(len(table)))

# Social Services and Social Work Only:
# Number of total Jobs:
# 	81817
# Number of Jobs with MA or Above:
# 	25126
# Number of non-faculty Jobs:
# 	17701

# Adding Policy Analysis, Policy Reserach, Social Studies, Policy Evaluation, and Policy Development
# Number of total Jobs:
# 	81817
# Number of Jobs with MA or Above:
# 	25126
# Number of non-faculty Jobs:
# 	18674


In [ ]:
pd.DataFrame(table[['Job ID']].merge(skill_table, on='Job ID', how='inner')['Skill Name'].value_counts())

In [ ]:
display(table[table['Year']==2017][['Career Area', 'Occupation', 'Job ID']].groupby([
    'Career Area', 'Occupation']).count())

In [ ]:
Regressison Analysis
Survey Research
Qualatative Research
Quantitative Research
Quantatiatve Analysis
Qualitative Analysis
Researh Design
Research

mask = skill_table

In [ ]:
clust_break['Skill Name'].value_counts()

# Social Policy Job Analysis

### The following visualizations are based on the space of jobs identified as being in the "Social Policy" sector based on a set of skills. Here are some examples of that skillspace:
##### Social Work
##### Social Services
##### Policy Analysis
##### Policy Reserach
##### Social Studies
##### Policy Evaluation
##### Policy Development
### All postings that request at least one of these Skills are then used as a the set of postings for further division. We are attemping to describe the Higher Education job space for those with PhDs in social policy or related fields. Therefore, the posting space is also limited to those that require a masters degree or greater.

In [ ]:
def get_categorical_sizes(df, category):
    df_size = len(df)
    cat = pd.DataFrame(df[category].value_counts()).reset_index().rename(
        columns={category:'count', 'index':category})
    cat['inc'] = np.true_divide(cat['count'], df_size)
    return cat

def get_colocated_skills(df, year=0):
    if year > 0:
        df = df[df['Year']==year]
        skill_source = skill_table[skill_table['Year']==year]
    else:
        skill_source = skill_table.copy()
    
    co_loc = pd.DataFrame(df[['Job ID']].merge(
        skill_source, on='Job ID', how='inner')['Skill Name'].value_counts())
    co_loc.columns = ['count']
    co_loc['inc'] = np.true_divide(co_loc['count'], len(df))
    return co_loc.reset_index()

def get_growth(df1, df2, category, cutoff = 30, min_rate=0.025):
    df1 = df1[df1['inc']>=min_rate]
    df2 = df2[df2['inc']>=min_rate]
    final_df = df1.merge(df2, on='index', how='outer', indicator = True)
    final_df = final_df[final_df['count_y']>=30]
    final_df['growth'] = np.true_divide(final_df['inc_y']-final_df['inc_x'], final_df['inc_x'])
    return final_df

def graph_categories(df, category, title, top=10):
    df = df[:top]

    rates = df['inc'].values
    names = df.iloc[:,0].values
    

    ind = np.array([x for x, _ in enumerate(names)])

    plt.bar(ind, rates, label='Incidence Rate')

    plt.xticks(ind, names, rotation = 'vertical')
    plt.xlabel(category + 's')
    plt.ylabel('Incidence Rate of\n' + category + 's in Social Policy')
    plt.legend(loc='upper right')
    plt.title(title)

    plt.show()

def graph_growth(df, year1, year2, category, title, min_rate=0.05, top=10):
    df = df[df['inc_y']>=min_rate]
    df = df[:top]
        
    rates = df['growth'].values
    names = df.iloc[:,0].values

    ind = np.array([x for x, _ in enumerate(names)])

    plt.bar(ind, rates, label='Growth rate')

    plt.xticks(ind, names, rotation = 'vertical')
    plt.xlabel(category + 's')
    plt.ylabel('Growth Rate of\n' + category + ' from ' + str(year1) + ' to ' + str(year2))
    plt.legend(loc='upper right')
    plt.title(title)

    plt.show()
    
def show_added(df, top=10, min_rate = 0.025):
    df = df[df['_merge']=='right_only']
    df = df[['index', 'count_y', 'inc_y']]
    df = df[df['inc_y']>=min_rate]
    df = df[:top]
    display(df)


In [ ]:
# Categorical Breakdowns
def occupational_breakdowns(df, category, year1, year2, faculty):

    years = val_year(year1, year2)
    year1 = years[0]
    yaer2 = years[1]

    df = df[(df['Year']==year1)|(df['Year']==year2)]

    if not faculty:
        print('Growth of Non-Faculty Social Policy Jobs from', year1, 'to', year2)
    else:
        print('Growth of Faculty Social Policy Jobs from', year1, 'to', year2)
    print()

    skill_2nd = ''

    if (skill_2nd != ''):
        mask = skill_table[skill_table['Skill Name']==skill_2nd][['Job ID']]
        df = df.merge(mask, on='Job ID', how='inner')

#     def get_categorical_sizes(df, category):
#         df_size = len(df)
#         cat = pd.DataFrame(df[category].value_counts()).reset_index().rename(
#             columns={category:'count', 'index':category})
#         cat['inc'] = np.true_divide(cat['count'], df_size)
#         return cat

    cat_1 = get_categorical_sizes(df[df['Year']==year1], category)

    cat_2 = get_categorical_sizes(df[df['Year']==year2], category)

    graph_categories(cat_1, category, title='Top ' + category + 's in ' + str(year1))

    graph_categories(cat_2, category, title='Top ' + category + 's in ' + str(year2))

    cat = cat_1.merge(cat_2, on=category, how='inner')
    cat = cat[cat['count_y']>=30]
    cat['growth'] = round(np.true_divide(cat['inc_y']-cat['inc_x'], cat['inc_x'])*100, 2)
    cat = cat.sort_values(by='growth', ascending=False)
    cat = cat.reset_index(drop=True)
#     cat = cat.rename(columns={'count_x':'count_'+str(year1), 'inc_x':'inc_'+str(year1),
#                               'count_y':'count_'+str(year2), 'inc_y':'inc_'+str(year2)})
    return cat


In [ ]:
# Breakdowns of Faculty Occupational Information
cat = occupational_breakdowns(table_fac, 'Occupation', 2010, 2017, faculty=False)
# display(cat)
graph_growth(cat, 2010, 2017, 'Occupation', 'Growth of Faculty Social Policy Occupations from '
             + str(year1) + ' to ' + str(year2), min_rate = 0.01)

In [ ]:
# Breakdowns of Non-Faculty Occupational Information
cat = occupational_breakdowns(table_non, 'Occupation', 2010, 2017, faculty=False)
# display(cat)
graph_growth(cat, 2010, 2017, 'Occupation', 'Growth of non-Faculty Social Policy Occupations from '
             + str(year1) + ' to ' + str(year2), min_rate = 0.01)

In [ ]:
year1 = 2010
year2 = 2017

years = val_year(year1, year2)
year1 = years[0]
year2 = years[1]

print('Colocated Faculty Skills in ' + str(year1) + ' and ' + str(year2))
fac_1 = get_colocated_skills(table_fac, year1)
fac_2 = get_colocated_skills(table_fac, year2)
graph_categories(fac_2, 'Skill Name', 'Top 10 Faculty Skills in ' + str(year2))
fac_grown = get_growth(fac_1, fac_2, 'Skill Name')
fac_grown = fac_grown.sort_values('growth', ascending=False)
graph_growth(fac_grown, year1, year2, 'Skill Name', 'Fastest Growing Faculty Co-Located Skills from '
             + str(year1) + ' to ' + str(year2))

print('Top Added Skills in Faculty:')
show_added(fac_grown)

print('Colocated Non-Faculty Skills in ' + str(year1) + ' and ' + str(year2))
non_1 = get_colocated_skills(table_non, year1)
non_2 = get_colocated_skills(table_non, year2)
graph_categories(fac_2, 'Skill Name', 'Top 10 Non-Faculty Skills in ' + str(year2))
non_grown = get_growth(non_1, non_2, 'Skill Name')
non_grown = non_grown.sort_values('growth', ascending=False)
graph_growth(non_grown, year1, year2, 'Skill Name', 'Fastest Growing Non-Faculty Co-Located Skills from '
             + str(year1) + ' to ' + str(year2))

print('Top Added Skills in Non-Faculty:')
show_added(non_grown)
# display(non_grown[non_grown['_merge']=='right_only'])


In [ ]:
# Break out Occupations of particular interest in the non_faculty sector
occ_break = table_non[(table_non['Occupation']=='Administrator, Higher Education')|
      (table_non['Occupation']=='Librarian')|
      (table_non['Occupation']=='Research Associate')|
      (table_non['Occupation']=='Dean of Academic Division')|
      (table_non['Occupation']=='Director of Student Affairs')|
      (table_non['Occupation']=='Dean of Academic Affairs')|
      (table_non['Occupation']=='Program Director, Higher Education')|
      (table_non['Occupation']=='Administrative Manager')|
      (table_non['Occupation']=='Dean of Students')|
      (table_non['Occupation']=='General Manager')|
      (table_non['Occupation']=='Policy Analyst')|
      (table_non['Occupation']=='Education Program Manager')|
      (table_non['Occupation']=='Academic / Guidance Counselor')|
      (table_non['Occupation']=='Associate Dean, Higher Education')|
      (table_non['Occupation']=='Social Science Researhcer')|
      (table_non['Occupation']=='Program Manager (General)')|
      (table_non['Occupation']=='Research Scienteist')|
      (table_non['Occupation']=='Curriculum and Instructional Designer')|
      (table_non['Occupation']=='Admissions Director')|
      (table_non['Occupation']=='Compliance Manager')|
      (table_non['Occupation']=='Fundraising Manager')|
      (table_non['Occupation']=='Human Reseources Specialist')|
      (table_non['Occupation']=='Student Affairs Administrator')|
      (table_non['Occupation']=='Grants Manager / Administrator')]

In [ ]:
# Special Bucket of 'Administrative/Academic' type jobs
category = 'Occupation'
cat = get_categorical_sizes(occ_break, category)
graph_categories(cat, category, title='Top ' + category + 's in Special Bucket')
category = 'Skill Name'
co_loc = get_colocated_skills(occ_break)
graph_categories(co_loc, category, title='Top ' + category + 's in Special Bucket')

schools = occ_break['IPEDS Institution Name'].nunique()
total = main_table['IPEDS Institution Name'].nunique()
print('Selected Non-Faculty Occupations appear in ' + str(schools) + 
      ' different institutions across the sample.\n')
print('This represents ' + str(round((schools/total)*100, 2))
      + '% of the institutions within the dataset.\n')
print('Most of the postings come from 4-Year institutions with many coming from R1 Institutions.')

display(occ_break[['R1', 'Public', 'Private', '4-year', '2-year', 'Job ID']].groupby(['R1', 'Public', 'Private', '4-year', '2-year']).count())


In [ ]:
main_table['IPEDS Institution Name'].nunique()

In [ ]:
829/4680